In [1]:
import pyscf
import pyscf.tools
from pyscf import lo, gto


from orbitalpartitioning import *


molecule = """
Cr -1.32078 0.00005 -0.00007
Cr 1.32077 0.00005 -0.00007
O 0.00000 -0.16583 1.45468
O 0.00000 1.34277 -0.58372
O 0.00000 -1.17683 -0.87101
H 0.00002 0.50128 2.15993
H 0.00056 1.61869 -1.51448
H -0.00044 -2.12079 -0.64413
N -2.64980 -1.44569 0.71142
H -2.18696 -2.18198 1.24440
H -3.05396 -1.84420 -0.13607
H -3.36727 -1.00512 1.28721
N -2.64980 1.33902 0.89630
N -2.64980 0.10677 -1.60777
H -3.36727 -0.61216 -1.51411
H -3.05396 0.80432 1.66516
N 2.64980 -1.44568 0.71142
N 2.64979 1.33903 0.89630
N 2.64980 0.10678 -1.60777
H -2.18697 2.16873 1.26745
H -3.36727 1.61737 0.22686
H -2.18696 0.01334 -2.51190
H -3.05397 1.03998 -1.52914
H 2.18696 -2.18197 1.24440
H 3.05396 -1.84419 -0.13608
H 3.36727 -1.00510 1.28720
H 2.18695 2.16874 1.26745
H 3.05396 0.80433 1.66516
H 3.36726 1.61738 0.22685
H 2.18696 0.01335 -2.51190
H 3.05396 1.03999 -1.52914
H 3.36727 -0.61215 -1.51411
"""
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   False,
        spin    =   6, # number of unpaired electrons
        charge  =   3)
pymol.build()

mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "minao"
mf.run()
##################################################################




******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 63  num. singly occ = 6
init E= -2620.13767269035
  HOMO = -0.166889672554435  LUMO = -0.073157679964048
cycle= 1 E= -2620.62812076959  delta_E= -0.49  |g|= 0.89  |ddm|= 4.88
  HOMO = -0.251286895044215  LUMO = 0.0571632846408885
cycle= 2 E= -2620.95219539261  delta_E= -0.324  |g|= 0.208  |ddm|=  1.2
  HOMO = -0.37610879291082  LUMO = 0.0119193166566399
cycle= 3 E= -2620.96767187302  delta_E= -0.0155  |g|= 0.0492  |ddm|= 0.258
  HOMO = -0.409037413345151  LUMO = -0.0135598591844162
cycle= 4 E= -2620.96878852075  delta_E= -0.00112  |g|= 0.0119  |ddm|= 0.083
  HOMO = -0.403412312824071  

ROHF-RHF object of <class 'pyscf.scf.rohf.ROHF'>

# ROHF/6-31G*

In [2]:
pymol.basis = "6-31g*"
pymol.build()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "chkfile"
mf.run()
##################################################################



******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = chkfile
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 63  num. singly occ = 6


In [ ]:
#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]

ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]

print(" Norbs = %6i ndocc = %6i nsing = %6i nvirt = %6i"%(Cdocc.shape[0], ndocc, nsing, nvirt))

 Norbs =    190 ndocc =     77 nsing =     10 nvirt =    103


In [ ]:
# Find AO's corresponding to atoms
doccs = []
frags = [[] for i in range(5)]
virts = []

for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    atom_number = ao[0]
    orbital = ao[2]
    if atom_number == 0:
        if orbital in ("3s", "3p"):
            doccs.append(ao_idx)
        if orbital in ("3d","4d"):
            frags[0].append(ao_idx)
        if orbital in ("4s", "4p"):
            virts.append(ao_idx)
    if atom_number == 2:
        if orbital in ("2s"):
            doccs.append(ao_idx)
        if orbital in ("2p","3s","3p"):
            frags[2].append(ao_idx)
        if orbital in ("3d"):
            virts.append(ao_idx)
    if atom_number == 1:
        if orbital in ("3s", "3p"):
            doccs.append(ao_idx)
        if orbital in ("3d","4d"):
            frags[1].append(ao_idx)
        if orbital in ("4s", "4p"):
            virts.append(ao_idx)
    if atom_number == 3:
        if orbital in ("2s"):
            doccs.append(ao_idx)
        if orbital in ("2p","3s","3p"):
            frags[3].append(ao_idx)
        if orbital in ("3d"):
            virts.append(ao_idx)
    if atom_number == 4:
        if orbital in ("2s"):
            doccs.append(ao_idx)
        if orbital in ("2p","3s","3p"):
            frags[4].append(ao_idx)
        if orbital in ("3d"):
            virts.append(ao_idx)


full = []
for f in frags:
    for fi in f:
        full.append(fi)

active = [i for i in full] 
full.extend(doccs)
full.extend(virts)

#
#   Get full active space
(Oact, Sact, Vact), (Oenv, Cerr, Venv) = svd_subspace_partitioning_nonorth((Cdocc, Csing, Cvirt), full, S)
assert(Cerr.shape[1] == 0)

#
#   Get full active space
(Oact, Sact, Vact), (Ocluster, Cerr, Vcluster) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), active, S)
assert(Cerr.shape[1] == 0)


 In svd_subspace_partitioning_nonorth
 Partition  190 orbitals into a total of   49 orbitals
            Index   Sing. Val. Space       
                0   0.99999718            2*
                1   0.99999698            0*
                2   0.99999545            2*
                3   0.99999507            0*
                4   0.99998867            0*
                5   0.99998785            0*
                6   0.99998411            0*
                7   0.99998367            0*
                8   0.99998155            0*
                9   0.99998086            0*
               10   0.99989780            0*
               11   0.99985233            2*
               12   0.99985217            2*
               13   0.99967182            0*
               14   0.99951803            2*
               15   0.99940574            2*
               16   0.99928479            0*
               17   0.99928229            0*
               18   0.99907943            2*
        

In [ ]:
#
#   Split active space into fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), f, S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print(" sing vals of active space: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
# for i in range(len(frags)):
#     di = Cfrags[i] @ Cfrags[i].T
#     pyscf.tools.cubegen.density(mf.mol, 'fragden_{:02d}.cube'.format(i+1), di)


# add doccs and virts. We'll add the doccs first, then the virts

init_fspace.append((Ocluster.shape[1], Ocluster.shape[1]))
init_fspace.append((0,0))

tmpi = Cact.shape[1]+1
tmpj = Ocluster.shape[1] 
tmpk = Vcluster.shape[1] 
clusters.append([i for i in range(tmpi,tmpi+tmpj)])
clusters.append([i for i in range(tmpi+tmpj,tmpi+tmpj+tmpk)])

Cact = np.hstack((Cact, Ocluster, Vcluster))


print("init_fspace = ", init_fspace)
print("clusters    = ", clusters)



 Fragment:  [17, 18, 19, 20, 21]
 In svd_subspace_partitioning_nonorth
 Partition   17 orbitals into a total of    5 orbitals
            Index   Sing. Val. Space       
                0   0.92578269            1*
                1   0.92360783            1*
                2   0.91561521            1*
                3   0.90711254            1*
                4   0.89646121            1*
                5   0.21961587            0
                6   0.14292561            0
                7   0.13420994            0
                8   0.06532186            2
                9   0.06247465            2
               10   0.00599335            2
               11   0.00084489            0
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              4              0
              1              5              5
              2              3              0

 Fragment:  [36, 37, 38, 39, 40, 41, 42]
 In svd_subspace_part

# Make integrals

In [ ]:
#   Make integrals
#
d1_embed = 2 * Oenv @ Oenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)